# Model Class

This notebook contains the AI model to finetune and generate the essay.

## Start Notebook
Run all of the command below to start the notebook training session of the model.

Uncomment and run the code below to kill the runtime in Google Colaboratory

In [ ]:
# !kill -9 -1


Uncomment and run the code below to install necessary file in Colaboratory

In [ ]:
# %tensorflow_version 1.x

# !pip install gpt_2_simple
# !pip install wikipedia

# !nvidia-smi


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')


In [ ]:
# !mkdir -p drive/My\ Drive/Project\ Writer/datasets
# !mkdir -p drive/My\ Drive/Project\ Writer/checkpoint
# !mkdir -p drive/My\ Drive/Project\ Writer/samples


Uncomment and run the code below to download the model in Colaboratory

In [ ]:
# import gpt_2_simple as gpt2
# gpt2.download_gpt2(model_name='355M')


Import necessary libraries

## Model
This class is used to finetune the model and generate text based on the current latest dataset. 

In [ ]:
class Model():
    '''

    Class to finetune the model and generate the text based on the dataset.

    @sess: GPT-2 TensorFlow session
    @model_name: The name of the model: 124M, 355M, etc.
    @run_name: The name of the trained model for each run
    @model_dir: The path to the directory containing the model
    @checkpoint_dir: The path to the directory containing the previously trained model
    @sample_dir: The path to the directory to save the sample
    @steps: Number of iteration to train the model
    @learning_rate: The rate of learning of the model

    Methods:


    '''

    # Import libraries
    import gpt_2_simple as gpt2
    import tensorflow as tf

    def __init__(
        self,
        sess,
        model_name,
        run_name,
        model_dir,
        checkpoint_dir,
        sample_dir,
        steps=30000,
        learning_rate=0.0001
    ):
        self.sess = sess
        self.model_name = model_name
        self.run_name = run_name
        self.model_dir = model_dir
        self.checkpoint_dir = checkpoint_dir
        self.sample_dir = sample_dir
        self.steps = steps
        self.learning_rate = learning_rate
    
    def finetune_model(self, dataset):
        '''
        
        Function to finetune the model and save the trained model every checkpoint on the checkpoint folder.

        @sess: Tensorflow training session
        @dataset: Path to the training dataset with minimum 1024 tokens

        return: None

        '''

        gpt2.finetune(
            self.sess,
            dataset=dataset,  # Dataset file
            steps=self.steps,
            model_name=self.model_name,  # Model name: 124M, 355M, etc.
            model_dir=self.model_dir,
            combine=50000,
            batch_size=1,
            learning_rate=self.learning_rate,  # Learning rate
            accumulate_gradients=5,
            restore_from='latest',  # Start training the model from the latest model
            run_name=self.run_name,  # Name of the trained model
            checkpoint_dir=self.checkpoint_dir,  # Directory to save the model
            sample_every=2500,
            sample_length=300,  # Number of token generated
            sample_num=1,
            multi_gpu=False,
            save_every=2500,
            print_every=100,
            max_checkpoints=1,
            use_memory_saving_gradients=False,
            only_train_transformer_layers=False,
            optimizer='adam',
            overwrite=True  # Overwrite the current model when training
        )

    def generate_text(self, outline_to_length):
        '''

        Function to generate the text.

        @outline_to_length: A 2D array containing the list of outline and the length desired
            [[outline, length],
            [outline, length],
            [outline, length]]

        return: List of generated text

        '''

        # Create an empty list to store lists
        essay = []

        # Loop over the list
        for record in outline_to_length:
            prefix = record[0]  # The first sentence of the paragraph
            length = record[1]  # The length of the paragraph (max: 1023)

            text = gpt2.generate(
                self.sess,
                run_name=self.run_name,
                checkpoint_dir=self.checkpoint_dir,
                model_name=None,
                model_dir=self.model_dir,
                sample_dir=self.sample_dir,
                return_as_list=True,  # Return as list of string
                truncate=None,
                destination_path=None,
                sample_delim='\n\n' + '=' * 10 + '\n\n',
                prefix=prefix,
                seed=None,
                nsamples=1,  # Number of sample to be generated
                batch_size=1,
                length=length,  # Length of the generated text max: 1024 tokens
                temperature=0.7,
                top_k=0,
                top_p=0.0,
                include_prefix=True
            )

            text = ''.join(text) + '\n\n'
            essay += text

        return essay

    def load_model(self):
        '''

        Function to load existing GPT-2 model.

        return: None

        '''

        gpt2.load_gpt2(
            self.sess,
            run_name=self.run_name,
            checkpoint_dir=self.checkpoint_dir,
            model_name=None,
            model_dir=self.model_dir,
            multi_gpu=False,
            reuse=True
        )

    def reset_session(self):
        '''
        
        Function to reset the current TensorFlow session, to clear memory or load another model.

        return: Reset session

        '''

        sess = gpt2.reset_session(self.sess)
        return sess



In [12]:
def finetune_model(dataset, any_checkpoint=False, reset_session=False):
    '''

    Function to finetune the model and save the trained model every checkpoint on the checkpoint folder.

    @dataset: Path to the training data (TXT) with minimum 1024 tokens
    @any_checkpoint: Boolean if there is any previous checkpoint
    @reset_session: Boolean if reseting the session graph is needed

    return: None

    '''

    # Parameters
    STEPS = 10000
    MODEL_NAME = '355M'
    LEARNING_RATE = 0.0001
    RUN_NAME = 'trained_model'

    MODEL_DIR = 'models'
    CHECKPOINT_DIR = 'checkpoint'

    # Clear session graph
    if reset_session:
        tf.reset_default_graph()

    # Initialize training session
    sess = gpt2.start_tf_sess()

    # Load the previous checkpoint
    if any_checkpoint:
        gpt2.load_gpt2(
            sess,
            run_name=RUN_NAME,
            checkpoint_dir=CHECKPOINT_DIR,
            model_name=None,
            model_dir=MODEL_DIR,
            multi_gpu=False
        )

    # Finetune the model
    gpt2.finetune(
        sess,
        dataset=dataset,  # Dataset file
        steps=STEPS,
        model_name=MODEL_NAME,  # Model name: 124M, 355M, etc.
        model_dir=MODEL_DIR,
        combine=50000,
        batch_size=1,
        learning_rate=LEARNING_RATE,  # Learning rate
        accumulate_gradients=5,
        restore_from='latest',  # Start training the model from the latest model
        run_name=RUN_NAME,  # Name of the trained model
        checkpoint_dir=CHECKPOINT_DIR,  # Directory to save the model
        sample_every=1000,
        sample_length=300,  # Number of token generated
        sample_num=1,
        multi_gpu=False,
        save_every=1000,
        print_every=10,
        max_checkpoints=1,
        use_memory_saving_gradients=False,
        only_train_transformer_layers=False,
        optimizer='adam',
        overwrite=True  # Overwrite the current model when training
    )


Uncomment and run the code below to test the finetuning model function

In [ ]:
# test_training_data = ''
# finetune_model(test_training_data)


## Generating Text
This functions is used to generate the text based on some input from the users.

In [ ]:
def generate_text(outline_to_length):
    '''

    Function to generate the text.

    @outline_to_length: A 2D array containing the list of outline and the length desired
        [[outline, length],
        [outline, length],
        [outline, length]]

    return: List of generated text

    '''

    # Parameters
    MODEL_NAME = '355M'
    RUN_NAME = 'trained_model'

    MODEL_DIR = 'models'
    CHECKPOINT_DIR = 'checkpoint'
    SAMPLE_DIR = 'samples'

    # Clear session graph
    tf.reset_default_graph()

    # Initialize TensorFlow session
    sess = gpt2.start_tf_sess()

    # Create an empty list to store lists
    essay = []

    # Loop over the list
    for record in outline_to_length:
        prefix = record[0]  # The first sentence of the paragraph
        length = record[1]  # The length of the paragraph (max: 1023)

        text = gpt2.generate(
            sess,
            run_name=RUN_NAME,
            checkpoint_dir=CHECKPOINT_DIR,
            model_name=None,
            model_dir=MODEL_DIR,
            sample_dir=SAMPLE_DIR,
            return_as_list=True,  # Return as list of string
            truncate=None,
            destination_path=None,
            sample_delim='\n\n' + '=' * 20 + '\n\n',
            prefix=prefix,
            seed=None,
            nsamples=1,  # Number of sample to be generated
            batch_size=1,
            length=length,
            temperature=0.7,
            top_k=0,
            top_p=0.0,
            include_prefix=True
        )

        text = ''.join(text) + '\n\n'
        essay += text

    return essay


Uncomment and run the code below to test the function

In [ ]:
# test_outline_to_length = [[]]
# print(generate_text(test_outline_to_length))
